## Setup

In [2]:
from dotenv import load_dotenv

load_dotenv()
llm_config={"model": "gpt-4o-mini"}

In [3]:
task = '''
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''


## Writer agent

In [4]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [5]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [6]:
print(reply)

**Unlocking the Future with DeepLearning.AI**

DeepLearning.AI is at the forefront of artificial intelligence education, revolutionizing how we learn about deep learning. Founded by AI visionary Andrew Ng, the platform offers comprehensive courses that cater to both beginners and advanced practitioners. With a hands-on approach and real-world applications, learners gain valuable skills that are highly sought after in today’s data-driven landscape. From foundational concepts to specialized topics, DeepLearning.AI empowers individuals and organizations to harness the potential of AI for innovation. Dive into the world of deep learning today and be part of the technological transformation shaping our future!


## Adding reflection 

In [8]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [9]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

**Unlocking the Future with DeepLearning.AI**

DeepLearning.AI is at the forefront of artificial intelligence education, revolutionizing how we learn about deep learning. Founded by AI visionary Andrew Ng, the platform offers comprehensive courses that cater to both beginners and advanced practitioners. With a hands-on approach and real-world applications, learners gain valuable skills that are highly sought after in today’s data-driven landscape. From foundational concepts to specialized topics, DeepLearning.AI empowers individuals and organizations to harness the potential of AI for innovation. Dive into the world of deep learning today and be part of the technological transformation shaping our future!

------------------------

## Nested chat

In [10]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [11]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [12]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [13]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

In [14]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [15]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

In [19]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

**Unlocking the Future with DeepLearning.AI**

DeepLearning.AI is at the forefront of artificial intelligence education, revolutionizing how we learn about deep learning. Founded by AI visionary Andrew Ng, the platform offers comprehensive courses that cater to both beginners and advanced practitioners. With a hands-on approach and real-world applications, learners gain valuable skills that are highly sought after in today’s data-driven landscape. From foundational concepts to specialized topics, DeepLearning.AI empowers individuals and organizations to harness the potential of AI for innovation. Dive into the world of deep learning today and be part of the technological transformation shaping our future!

------------------------

In [20]:
import pprint
pprint.pprint(res.summary)

('**Unlocking the Future with DeepLearning.AI**\n'
 '\n'
 'DeepLearning.AI is revolutionizing AI education with its in-depth deep '
 'learning courses designed by industry leader Andrew Ng. These courses offer '
 'an engaging, hands-on experience suitable for everyone—from beginners to '
 'advanced practitioners. \n'
 '\n'
 'Whether you want to explore foundational concepts or specialized techniques, '
 'DeepLearning.AI provides the skills necessary to excel in today’s '
 'data-driven landscape. Accessible and inclusive, the platform prioritizes '
 'diverse learning styles and user privacy. \n'
 '\n'
 'Join the AI revolution today! Enroll now to transform your career and '
 'leverage the power of deep learning. Learn more about our courses and start '
 'your journey into the future of technology!')


In [21]:
res.cost

{'usage_including_cached_inference': {'total_cost': 0.0013557,
  'gpt-4o-mini-2024-07-18': {'cost': 0.0013557,
   'prompt_tokens': 3098,
   'completion_tokens': 1485,
   'total_tokens': 4583}},
 'usage_excluding_cached_inference': {'total_cost': 0.0006593999999999999,
  'gpt-4o-mini-2024-07-18': {'cost': 0.0006593999999999999,
   'prompt_tokens': 1488,
   'completion_tokens': 727,
   'total_tokens': 2215}}}